# Custom solvers
In this example, we show how to define custom solvers. Our system
will again be silicon, because we are not very imaginative

In [1]:
using DFTK
using LinearAlgebra
using PseudoPotentialData
using AtomsBuilder

# We take very (very) crude parameters
pseudopotentials = PseudoFamily("dojo.nc.sr.lda.v0_4_1.standard.upf")
model = model_DFT(bulk(:Si); functionals=LDA(), pseudopotentials)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[1, 1, 1]);

We define our custom fix-point solver: simply a damped fixed-point

In [2]:
function my_fp_solver(f, x0, info0; maxiter)
    mixing_factor = .7
    x = x0
    info = info0
    for n = 1:maxiter
        fx, info = f(x, info)
        if info.converged || info.timedout
            break
        end
        x = x + mixing_factor * (fx - x)
    end
    (; fixpoint=x, info)
end;

Note that the fixpoint map `f` operates on an auxiliary variable `info` for
state bookkeeping. Early termination criteria are flagged from inside
the function `f` using boolean flags `info.converged` and `info.timedout`.
For control over these criteria, see the `is_converged` and `maxtime`
keyword arguments of `self_consistent_field`.

Our eigenvalue solver just forms the dense matrix and diagonalizes
it explicitly (this only works for very small systems)

In [3]:
function my_eig_solver(A, X0; maxiter, tol, kwargs...)
    n = size(X0, 2)
    A = Array(A)
    E = eigen(A)
    λ = E.values[1:n]
    X = E.vectors[:, 1:n]
    (; λ, X, residual_norms=[], n_iter=0, converged=true, n_matvec=0)
end;

Finally we also define our custom mixing scheme. It will be a mixture
of simple mixing (for the first 2 steps) and than default to Kerker mixing.
In the mixing interface `δF` is $(ρ_\text{out} - ρ_\text{in})$, i.e.
the difference in density between two subsequent SCF steps and the `mix`
function returns $δρ$, which is added to $ρ_\text{in}$ to yield $ρ_\text{next}$,
the density for the next SCF step.

In [4]:
struct MyMixing
    n_simple  # Number of iterations for simple mixing
end
MyMixing() = MyMixing(2)

function DFTK.mix_density(mixing::MyMixing, basis, δF; n_iter, kwargs...)
    if n_iter <= mixing.n_simple
        return δF  # Simple mixing -> Do not modify update at all
    else
        # Use the default KerkerMixing from DFTK
        DFTK.mix_density(KerkerMixing(), basis, δF; kwargs...)
    end
end

That's it! Now we just run the SCF with these solvers

In [5]:
scfres = self_consistent_field(basis;
                               tol=1e-4,
                               solver=my_fp_solver,
                               eigensolver=my_eig_solver,
                               mixing=MyMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.844704367369                   -0.58    0.0    1.52s
  2   -7.856031172216       -1.95       -1.01    0.0    1.32s
  3   -7.857206749077       -2.93       -1.43    0.0   56.0ms
  4   -7.857296337864       -4.05       -1.75    0.0   88.6ms
  5   -7.857319136840       -4.64       -2.06    0.0    109ms
  6   -7.857325084669       -5.23       -2.36    0.0   55.9ms
  7   -7.857326696758       -5.79       -2.66    0.0   65.3ms
  8   -7.857327153270       -6.34       -2.95    0.0    166ms
  9   -7.857327288370       -6.87       -3.23    0.0   67.2ms
 10   -7.857327330007       -7.38       -3.49    0.0   68.4ms
 11   -7.857327343295       -7.88       -3.75    0.0    619ms
 12   -7.857327347658       -8.36       -4.00    0.0   56.4ms


Note that the default convergence criterion is the difference in
density. When this gets below `tol`, the fixed-point solver terminates.
You can also customize this with the `is_converged` keyword argument to
`self_consistent_field`, as shown below.

## Customizing the convergence criterion
Here is an example of a defining a custom convergence criterion and specifying
it using the `is_converged` callback keyword to `self_consistent_field`.

In [6]:
function my_convergence_criterion(info)
    tol = 1e-10
    length(info.history_Etot) < 2 && return false
    ΔE = (info.history_Etot[end-1] - info.history_Etot[end])
    ΔE < tol
end

scfres2 = self_consistent_field(basis;
                                solver=my_fp_solver,
                                is_converged=my_convergence_criterion,
                                eigensolver=my_eig_solver,
                                mixing=MyMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.844704367369                   -0.58    0.0    138ms
  2   -7.856031172216       -1.95       -1.01    0.0    141ms
  3   -7.857206749077       -2.93       -1.43    0.0   68.0ms
  4   -7.857296337864       -4.05       -1.75    0.0   93.7ms
  5   -7.857319136840       -4.64       -2.06    0.0   56.3ms
  6   -7.857325084669       -5.23       -2.36    0.0   56.6ms
  7   -7.857326696758       -5.79       -2.66    0.0    147ms
  8   -7.857327153270       -6.34       -2.95    0.0   54.6ms
  9   -7.857327288370       -6.87       -3.23    0.0   56.5ms
 10   -7.857327330007       -7.38       -3.49    0.0   58.1ms
 11   -7.857327343295       -7.88       -3.75    0.0   61.9ms
 12   -7.857327347658       -8.36       -4.00    0.0   68.2ms
 13   -7.857327349123       -8.83       -4.25    0.0    117ms
 14   -7.857327349624       -9.30       -4.48    0.0   56.3ms
 15   -7.